# Spark Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [2]:
import findspark

In [3]:
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7/')

In [4]:
import pyspark

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('rec').getOrCreate()

With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [12]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [13]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [14]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [15]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [16]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [18]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [19]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [20]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|     5|-0.77443343|
|     85|   1.0|    28|   1.142145|
|     85|   1.0|    12| -2.0638514|
|     85|   3.0|     1|  2.8376107|
|     85|   2.0|    20|   4.429983|
|     85|   1.0|     5| -1.3630326|
|     85|   1.0|    25| -1.3778033|
|     65|   1.0|    28| -1.4966046|
|     65|   1.0|    22|  0.6142463|
|     65|   2.0|     5|  2.9484355|
|     65|   1.0|    19|   2.675062|
|     65|   1.0|    24|  1.5413449|
|     53|   1.0|    23|  1.3733267|
|     53|   1.0|     7|   2.361076|
|     78|   1.0|    19|  1.3000853|
|     34|   1.0|    28|   2.685715|
|     34|   1.0|    19| -0.1709607|
|     34|   1.0|    17|  0.8811735|
|     81|   3.0|    26|  1.8198261|
|     81|   2.0|     5|  2.6440444|
+-------+------+------+-----------+
only showing top 20 rows



In [21]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.9488922875169277


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [22]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [23]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      6|    11|
|     27|    11|
|     30|    11|
|     40|    11|
|     41|    11|
|     59|    11|
|     64|    11|
|     66|    11|
|     71|    11|
|     89|    11|
+-------+------+



In [24]:
reccomendations = model.transform(single_user)

In [25]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     27|    11|  2.913575|
|     30|    11|   2.74611|
|     89|    11| 2.3474643|
|     66|    11| 2.2243073|
|     71|    11| 1.9650327|
|     41|    11| 1.6263067|
|     59|    11| 0.7981719|
|     40|    11| 0.4030537|
|     64|    11|0.30503312|
|      6|    11|0.17293528|
+-------+------+----------+



# Great Job!